In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 9.1 MB/s eta 0:00:00


In [3]:
#import libraries
import numpy as np
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchmetrics

In [4]:
#Load the device GPU
gpu = torch.device("cuda:0")
#gpu = torch.device("cpu")
print(torch.cuda.get_device_name(torch.cuda.current_device()))

Tesla T4


In [5]:
#load the dataset
import codecs
dataset_t = ""
with codecs.open('/content/drive/Othercomputers/Il mio computer/BarberoGenerator/dataset/dataset_barbero_sarzana.txt', encoding='utf-8') as f:
    for character in f:
        dataset_t = dataset_t + character

In [6]:
# transform dataset from numeric to one-hot
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
whole_dataset_list_of_chars = list(dataset_t)
dataset_int = pd.DataFrame(whole_dataset_list_of_chars, columns=['data'])

# create a dictionary
label_enc = LabelEncoder()
dictionary = dataset_int.drop_duplicates(subset=['data'])
dictionary['int_encoding'] = label_enc.fit_transform(dictionary['data'])
dataset_int['int_encoding'] = label_enc.fit_transform(dataset_int['data'])


# one hot encode
one_hot_enc = OneHotEncoder()
# lstm uses float32 insted of float64.
one_hot_encoded_dataset = one_hot_enc.fit_transform(dataset_int[['int_encoding']]).toarray().astype(np.float32)
one_hot_encoded_dataset = torch.from_numpy(one_hot_encoded_dataset)
one_hot_encoded_dataset = one_hot_encoded_dataset.to(gpu)
print(one_hot_encoded_dataset)

<ipython-input-6-c0c5f10b0c7b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dictionary['int_encoding'] = label_enc.fit_transform(dictionary['data'])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')


In [7]:
print(dictionary)

        data  int_encoding
0          b            39
1          e            42
2          n            51
4                        1
6          u            58
...      ...           ...
1099082    -             4
1099618    U            33
1100751    È            64
1101278    N            26
1104174    V            34

[72 rows x 2 columns]


In [8]:
# check that the device set is the GPU
#assert one_hot_encoded_dataset.get_device() == 0

In [39]:
# ADD LABELS: turn dataset from a string "s1,s2,...,si,si+1" to ((s1,...,sk),(s2,...,sk+1)),...
sequence_length = 100
dataset = []
for i in range(0,len(one_hot_encoded_dataset)-sequence_length,int(sequence_length/2)):
    x = []
    y = []
    for j in range(sequence_length):
        x.append(one_hot_encoded_dataset[i+j])
        y.append(one_hot_encoded_dataset[i+j+1])
    dataset.append((x, y))
print(len(dataset))

22736


In [10]:
print(len(dataset[0]))

2


In [11]:
#create the dataloader: 70% for training, 30% for test
train_set_threshold = int(len(dataset) * 0.7)
training_set = dataset[:train_set_threshold]
test_set = dataset[train_set_threshold+1:]

In [18]:
batch_size_train = 64
batch_size_test = 128
train_dataloader = DataLoader(training_set, batch_size=batch_size_train, shuffle=False)
test_dataloader = DataLoader(test_set, batch_size=batch_size_test, shuffle=True)
iterator = iter(train_dataloader)
x, y = next(iter(iterator))
# check that the data is in the form: L,N,H. L=sequence length, N=batch size, H=input size
assert len(x) == sequence_length and len(x[0]) == batch_size_train and len(x[0][0]) == 72
assert len(y) == sequence_length and len(y[0]) == batch_size_train and len(y[0][0]) == 72
#for x,y in iterator:
#    print(x)
#    print("\n")

In [19]:
#build architecture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        # weights and biases are initialized using the uniform distribution:
        #      weights.uniform_(-sqrt(output_size), +sqrt(output_size))
        self.lstm_layer = torch.nn.LSTMCell(input_size=input_size, hidden_size=hidden_size)
        self.dense = torch.nn.Linear(in_features=hidden_size, out_features=output_size)

    def forward(self, x):
        outputs = []
        # we convert a list of tensors (given as such by the dataloader) to a tensor
        x = torch.stack(x)

        # the split function collapses the batch dimension, so now, instead of having (sequence_len,batch_size,input_size) inputs,
        # you have a number equal to batch_size of (batch_size,input_size) inputs; you will feed these inputs one after another to the lstm, using
        # the history h_t and short term memory c_t, along with the i-th input of type (batch_size,input_size)
        for time_step in torch.split(x, split_size_or_sections=1, dim=0):
            # this is needed to remove a dimension that wasn't removed by split
            time_step = torch.squeeze(time_step)
            h_t, c_t = self.lstm_layer(time_step)
            output = self.dense(h_t)
            outputs.append(output)
        # len(outputs) = batch_size. We convert outputs to a tensor by concatenating all tensors inside it
        #outputs = torch.stack(outputs)
        #print(outputs.size())
        return outputs
assert len(training_set[0][0][0]) == 72
input_size = len(training_set[0][0][0])
output_size = input_size
model = NeuralNetwork(input_size, 32, output_size)
model.to(gpu)
print(input_size)

72


In [20]:
# try the model
inp = next(iter(train_dataloader))[0]
output = model(inp)
print(len(output))
print(output[1].size())
#assert output.size() == (100,128,72)
print(torch.stack(output).size())

100
torch.Size([64, 72])
torch.Size([100, 64, 72])


In [21]:
#declare loss, optimizer
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
#train
def train_cycle(EPOCHS, train_dataloader, model, loss, optimizer):
    losses = []
    for i in range(EPOCHS):
        iterator = iter(train_dataloader)
        print("Epoch: " + str(i))
        for data, label in iterator:
            # pytorch accumulate gradients at every batch, doing this you reset these gradients
            optimizer.zero_grad()
            #forward step
            outputs = model(data)
            loss_acc = 0
            for i in range(len(outputs)):
                loss_value = loss(outputs[i],label[i])
                #backward step: compute gradients (apply automatic differentiation)
                loss_value.backward()
                loss_acc = loss_acc + loss_value
            loss_acc = loss_acc / len(outputs)
            losses.append(loss_acc)
            #update the parameters using the already computed gradients
            optimizer.step()
    return losses

In [24]:
import time

start = time.time()

EPOCHS = 2
losses = train_cycle(EPOCHS, train_dataloader, model, loss, optimizer)

end = time.time()
print("Time passed: " + str(end - start))
print(losses)

Epoch: 0
Epoch: 1
Time passed: 49.842140913009644
[tensor(4.2995, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2916, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2871, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2853, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2845, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2785, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2765, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2743, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2731, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2717, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2652, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2590, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2557, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2550, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2493, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2463, device='cuda:0', grad_fn=<DivBackward0>), tensor(4.2413, device='cuda:0', grad_

In [40]:
#test
#from torchmetrics.classification import MulticlassAccuracy
iterator = iter(test_dataloader)
#accuracy = MulticlassAccuracy(num_classes=72).to(gpu)
softmax = torch.nn.Softmax(dim=1)
accuracies = []
i = 0
for data, label in iterator:
    #reshape data and label from sequence_len x num_classes
    prediction = model(data)
    prediction = torch.stack(prediction)
    label = torch.stack(label)
    #compute accuracy
    pred_arg = torch.argmax(softmax(prediction),dim=2)
    label_arg = torch.argmax(label,dim=2)
    #print(torch.eq(pred_arg, label_arg).size())
    acc_list = (torch.sum(torch.eq(pred_arg, label_arg),1)/batch_size_test).tolist()
    mean_acc = sum(acc_list)/sequence_length
    accuracies.append(mean_acc)
print(accuracies)

[0.22453125, 0.2303125, 0.2203125, 0.223359375, 0.222109375, 0.220390625, 0.220546875, 0.2190625, 0.22203125, 0.225234375, 0.221796875, 0.226484375, 0.21828125, 0.22734375, 0.2284375, 0.226171875, 0.227578125, 0.218671875, 0.226015625, 0.22140625, 0.222578125, 0.224921875, 0.2253125, 0.23109375, 0.22515625, 0.221640625, 0.225078125, 0.22296875, 0.214140625, 0.22625, 0.22515625, 0.218125, 0.22640625, 0.217109375, 0.2275, 0.222421875, 0.218671875, 0.21625, 0.22140625, 0.222890625, 0.22484375, 0.2221875, 0.22203125, 0.21859375, 0.223515625, 0.221953125, 0.2240625, 0.22609375, 0.218515625, 0.2253125, 0.227421875, 0.225, 0.21859375, 0.060390625]


In [41]:
print("Mean accuracy is: " + str(sum(accuracies)/len(accuracies)))

Mean accuracy is: 0.22006799768518517


In [42]:
# this function encoded a string as an input fitting for the NN of size (SxN)
# * N = number of classes
# * S = length of the string (in terms of characters)
def generate_phrase_to_NN_input(num_classes, dictionary, device):
    # input: a string
    # output: a Sx72 one-hot tensor
    def phrase_to_NN_input(phrase):
        result = []
        for char in phrase:
            word_converted = torch.zeros(num_classes)
            word_index = dictionary.loc[dictionary['data'] == char]['int_encoding'].item()
            word_converted[word_index] = 1
            result.append(word_converted)
        result = torch.stack(result)
        result = result.to(device)
        return result
    return phrase_to_NN_input
phrase_to_NN_input = generate_phrase_to_NN_input(72,dictionary, gpu)

In [43]:
# this function decodes a string expressed as a tensor of shape SxN with values x\in{0,1}, to a string of length S
def generate_NN_output_to_phrase(num_classes, dictionary, device):
    # input: a string
    # output: a Sx72 one-hot tensor
    def NN_output_to_phrase(nn_output):
        result = ""
        for tensor_hot_enc in nn_output:
            idx_word = torch.argmax(tensor_hot_enc).item()
            char_decoded = dictionary.loc[dictionary['int_encoding'] == idx_word]['data'].item()
            result = result + char_decoded
        return result
    return NN_output_to_phrase
NN_output_to_phrase = generate_NN_output_to_phrase(72,dictionary, gpu)

In [44]:
wordminusminus_encoded = phrase_to_NN_input("--")
assert wordminusminus_encoded[0][4].item() == 1.0 and wordminusminus_encoded[1][4].item() == 1.0

In [45]:
wordminusminus_decoded = NN_output_to_phrase(wordminusminus_encoded)
assert wordminusminus_decoded == "--"

In [46]:
phrase = "eqweqweqwaaIII--------IIIIIaaaaaAAAAAAAAAAA"
phrase_encoded = phrase_to_NN_input(phrase)
prediction = model([phrase_encoded])
prediction_decoded = NN_output_to_phrase(prediction[0])
print(prediction_decoded)

 u  u  u                                   
